The purpose of the attack implemented is to retrieve the first round subkey used. Once enough of this subkey is determined, the full key used in the DES encryption can be found by exhaustive search methods.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.stats
import glob


In [ ]:
sys.path.append('gdrive/My Drive/DES_CPA/DES_Code')


In [ ]:
import des_block

In [ ]:
import matplotlib.pyplot as plt
import scipy.io as sp

!pip install -q bokeh

from bokeh.models import Range1d
from bokeh.plotting import figure, show
from bokeh.models import Range1d, ColorBar, LinearColorMapper, BasicTicker
from bokeh.io import output_notebook
import bokeh.colors.named as bokeh_colors_names

bokeh_colors_names_arr = dir(bokeh_colors_names)[10:]
bokeh_colors_names_arr = bokeh_colors_names_arr*3

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

In [ ]:
def getHeatMap(title, xaxis, yaxis, data, dh, dw):
    indMax = np.unravel_index(np.argmax(data, axis=None), data.shape)
    print(np.shape(data), data[indMax])    
    color_mapper = LinearColorMapper(palette="Turbo256", low=1e-2, high=data[indMax])
    #Log heatmap

    fig = figure(title=title, x_range=(0,dw), y_range=(0,dh), x_axis_label=xaxis, y_axis_label=yaxis,
        tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], plot_height=700, plot_width=700)
    fig.image(image=[data], x=0, y=0, dw=dw, dh=dh, color_mapper=color_mapper, level="image")
    #palette values-https://docs.bokeh.org/en/latest/docs/reference/palettes.html

    #add heat map
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(), border_line_color=None, location=(0,0))
    fig.add_layout(color_bar, 'right')
    #show(fig)
    return fig

In [ ]:
path_to_folder = '/content/gdrive/My Drive/DES_CPA/DES_Traces_Data'

plain_texts = []
power_traces_list = []
ciphers = []

all_files = glob.glob(f'{path_to_folder}/*.csv')

current_file_index = 0

for power_trace_file in all_files:
    folder, file = os.path.split(power_trace_file)

    file_no_extension = os.path.splitext(file)[0]

    plain_text_index = file_no_extension.index('m=') + 2

    plain_text = file_no_extension[plain_text_index:plain_text_index + 16]
    plain_texts.append(plain_text)
    cipher_index = file_no_extension.index('c=') + 2
    cipher_text = file_no_extension[cipher_index:cipher_index + 16]
    ciphers.append(cipher_text)

    power_traces_csv_contents = pd.read_csv(power_trace_file, header=None)

    current_power_traces = list(power_traces_csv_contents.iloc[:, 1])

    power_traces_list.append(current_power_traces)

    current_file_index+=1

    print(f'{current_file_index}/{len(all_files)}', end=" ")
    if (current_file_index % 20) == 19:
      print('\n')

power_traces = np.array(power_traces_list)



1/500 2/500 3/500 4/500 5/500 6/500 7/500 8/500 9/500 10/500 11/500 12/500 13/500 14/500 15/500 16/500 17/500 18/500 19/500 

20/500 21/500 22/500 23/500 24/500 25/500 26/500 27/500 28/500 29/500 30/500 31/500 32/500 33/500 34/500 35/500 36/500 37/500 38/500 39/500 

40/500 41/500 42/500 43/500 44/500 45/500 46/500 47/500 48/500 49/500 50/500 51/500 52/500 53/500 54/500 55/500 56/500 57/500 58/500 59/500 

60/500 61/500 62/500 63/500 64/500 65/500 66/500 67/500 68/500 69/500 70/500 71/500 72/500 73/500 74/500 75/500 76/500 77/500 78/500 79/500 

80/500 81/500 82/500 83/500 84/500 85/500 86/500 87/500 88/500 89/500 90/500 91/500 92/500 93/500 94/500 95/500 96/500 97/500 98/500 99/500 

100/500 101/500 102/500 103/500 104/500 105/500 106/500 107/500 108/500 109/500 110/500 111/500 112/500 113/500 114/500 115/500 116/500 117/500 118/500 119/500 

120/500 121/500 122/500 123/500 124/500 125/500 126/500 127/500 128/500 129/500 130/500 131/500 132/500 133/500 134/500 135/500 136/500 137/500 

In [ ]:
#  shrink it a little so it runs faster
power_traces = power_traces[:,0:8000] 
power_traces.shape

input_count = np.shape(power_traces)[0]
trace_length = np.shape(power_traces)[1]

In [ ]:
# These values can be changed to choose which trace to show
idx_1 = 0
idx_2 = 1

p = figure(title='Power traces of DES computation', x_axis_label='Time (ms)', y_axis_label='value', tooltips=[("x", "$x"), ("y", "$y")])
times = np.linspace(0,len(power_traces[0, :]), len(power_traces[0, :]))
p.x_range=Range1d(300, 500)
p.line(times, power_traces[idx_1, :], legend_label=f'Trace {idx_1}', line_color='blue')
p.line(times, power_traces[idx_2, :], legend_label=f'Trace {idx_2}', line_color='orange')
show(p)

In [ ]:
KEY_PARTS_COUNT = 8
trace_classification  = np.zeros(shape=(KEY_PARTS_COUNT, 2**6, input_count))
classification_output = np.zeros(shape=(KEY_PARTS_COUNT, 2**6, trace_length))
key_byte_to_correct_time = {}
key_byte_index_to_correct_byte = {}

In [ ]:
def get_hamming_weights(plain_texts: list, key_byte_to_guess: int):
    num_traces = len(plain_texts)
    guess_matrix = np.zeros((2 ** 6, num_traces))
    for key_byte_guess in range(2 ** 6):
        for trace_index in range(num_traces):
            msg = plain_texts[trace_index]

            ip = des_block.des_block(msg, 64).ip()
            l0 = ip.subblock(0, 32)
            r0 = ip.subblock(32, 64)
            e0 = r0.e().subblock(key_byte_to_guess * 6, (key_byte_to_guess + 1) * 6)
            s0 = l0.p(-1).subblock(key_byte_to_guess * 4, (key_byte_to_guess + 1) * 4)
            key_des_block = des_block.__from_int__(key_byte_guess, 6)
            r1_block = e0.xor(key_des_block).s(key_byte_to_guess).xor(s0)
            r0_block = r0.p(-1).subblock(key_byte_to_guess * 4, (key_byte_to_guess + 1) * 4)
            r0_r1_blocks_xor_hw = r0_block.xor(r1_block).hw()

            guess_matrix[key_byte_guess][trace_index] = r0_r1_blocks_xor_hw

    return guess_matrix

def print_graph(title, x_name, y_name, points, color='blue'):
    p = figure(title=title,
               x_axis_label=x_name, y_axis_label=y_name, tooltips=[("x", "$x"), ("y", "$y")])
    times = np.linspace(0,len(points), len(points))
    p.x_range=Range1d(0, len(points))
    p.line(times, points, line_color=color)
    show(p)

def extract_key_byte(plain_texts: list, power_traces: np.array, key_byte_to_guess: int, should_print_graphs=True):
    key_guess_hex_display = {'[{:02x}]'.format(key_byte_to_guess)}
    print(f'***** Extracting key byte {key_guess_hex_display}')

    hamming_weights = get_hamming_weights(key_byte_to_guess=key_byte_to_guess, plain_texts=plain_texts)
    correlation_per_key_byte = np.empty([hamming_weights.shape[0], power_traces.shape[1]])

    for key_guess in range(correlation_per_key_byte.shape[0]):
        trace_classification[key_byte_to_guess, key_guess, :] = hamming_weights[key_guess]
        for power_trace_time_index in range(power_traces.shape[1]):
            current_time_power_trace = power_traces[:, power_trace_time_index]

            current_hamming_weight = hamming_weights[key_guess, :]

            current_cor = scipy.stats.pearsonr(current_time_power_trace, current_hamming_weight)[0]

            correlation_per_key_byte[
                key_guess, power_trace_time_index] = current_cor
            classification_output[key_byte_to_guess, key_guess, power_trace_time_index] = current_cor

    index = np.unravel_index(np.argmax(correlation_per_key_byte, axis=None), correlation_per_key_byte.shape)
    correlated_key_guess = index[0]
    correlated_time = index[1]
    key_byte_to_correct_time[key_byte_to_guess] = correlated_time
    key_byte_index_to_correct_byte[key_byte_to_guess] = correlated_key_guess
    print(correlated_key_guess, correlated_time, np.max(correlation_per_key_byte, axis=None))

    if should_print_graphs:
          print_graph('The correlation between the power information and the sensitive data for each byte guess for the correct index',
                'Key Guess', 'Correlation', correlation_per_key_byte[:, correlated_time])
          print_graph('The correlation between the power information and the sensitive data for each time index for the correct byte guess',
                'Time index', 'Correlation', correlation_per_key_byte[correlated_key_guess, :], color='orange')

    return correlated_key_guess

turn_key = des_block.des_block()
for key_byte_to_guess in range(KEY_PARTS_COUNT):
    part_key_extracted = extract_key_byte(plain_texts=plain_texts, power_traces=power_traces,
                                          key_byte_to_guess=key_byte_to_guess)
    turn_key = turn_key.concat(des_block.__from_int__(part_key_extracted, 6))

# Computing uncomplete initial_key
cd1 = turn_key.pc2(-1)
c0 = cd1.subblock(0, 28).rs(1)
d0 = cd1.subblock(28, 56).rs(1)
uncomp_cd0 = c0.concat(d0)
ret_key = None

# Trying encipherement with the remaining 256 possible keys
for i in range(2**8):

    cd0 = uncomp_cd0.fill(des_block.__from_int__(i, 8))
    key = cd0.pc1(-1).fill(des_block.__from_int__(0, 8))
    cip = des_block.des_block(plain_texts[0], 64).encipher(key)
    if cip.value() == des_block.des_block(ciphers[0], 64).value():
        ret_key = key

print(f'Key is {ret_key}')

***** Extracting key byte {'[00]'}
56 5743 0.36443033725699175


***** Extracting key byte {'[01]'}
11 5231 0.3505598687854173


***** Extracting key byte {'[02]'}
59 5233 0.33767862614019634


***** Extracting key byte {'[03]'}
38 5741 0.24668944719962554


***** Extracting key byte {'[04]'}
0 5246 0.27749489245888015


***** Extracting key byte {'[05]'}
13 5752 0.38357693864230535


***** Extracting key byte {'[06]'}
25 5747 0.2861365191337701


***** Extracting key byte {'[07]'}
55 5753 0.3351954801333321


Key is 0x6a64786a64786a64


In [ ]:
#  Plot the trace classification matrix

# Which trace key byte index to show trace classification for, can be modified
# Value has to be between 0 and 7 (inclusive)
key_index = 0
trace_classification_for_idx = trace_classification[key_index, :, :]
[dh, dw] = np.shape(trace_classification_for_idx)

show(getHeatMap('Trace classification', 'Trace index', 'Key guess for byte  ' + str(key_byte_to_guess), trace_classification_for_idx, dh, dw))

(64, 500) 4.0


In [ ]:
# Which trace key byte index to show correlation for, can be modified
# Value has to be between 0 and 7 (inclusive)
key_guess_index = 0

correct_time = key_byte_to_correct_time[key_guess_index]
classification_output_for_key_index = classification_output[key_guess_index, :]
heights = np.abs(classification_output_for_key_index[:, correct_time])
p = figure(title='Correlation for each value of the byte', x_axis_label='Key guess', y_axis_label='Correlation', tooltips=[("x", "$x"), ("y", "$y")])
p.vbar(x=range(1, np.shape(classification_output_for_key_index)[0] + 1), top=heights, width=0.8, fill_color='blue')
show(p)

In [ ]:
# Which trace key byte index to show, can be modified.
# Value has to be between 0 and 7 (inclusive).
key_guess_index = 0
classification_output_for_key_index = classification_output[key_guess_index, :]


window_frame = 100#ms
offframe = 2*window_frame
ys=[x[correct_time - offframe:correct_time + offframe] for x in classification_output_for_key_index]
p = figure(title='The correct key at the correct time', x_axis_label='Time (ms)', y_axis_label='Correlation', tooltips=[("x", "$x"), ("y", "$y"), ("Key guess", "$index")])
times = np.linspace(correct_time - offframe, correct_time + offframe, offframe*2)
times = np.tile(times,(len(ys),1))#duplicate the array for the multiline function
p.multi_line(xs=list(times), ys=ys, color=bokeh_colors_names_arr[:len(times)])
#p.multi_line(xs=[[1, 2, 3], [2, 3, 4]], ys=[[6, 7, 2], [4, 5, 7]], color=['red','green'])
p.x_range=Range1d(correct_time - window_frame, correct_time + window_frame)
show(p)